In [ ]:
!unzip "/content/drive/MyDrive/Datasets/mms/spectrograms.zip"

Streaming output truncated to the last 5000 lines.
  inflating: spectrograms/transformed/sawing/file30.png  
  inflating: spectrograms/transformed/sawing/file35.png  
  inflating: spectrograms/transformed/sawing/file38.png  
  inflating: spectrograms/transformed/sawing/file54.png  
  inflating: spectrograms/transformed/sawing/file59.png  
  inflating: spectrograms/transformed/sawing/file62.png  
  inflating: spectrograms/transformed/sawing/file66.png  
  inflating: spectrograms/transformed/sawing/file75.png  
  inflating: spectrograms/transformed/sawing/file82.png  
  inflating: spectrograms/transformed/sawing/file85.png  
  inflating: spectrograms/transformed/sawing/file93.png  
  inflating: spectrograms/transformed/sawing/file96.png  
  inflating: spectrograms/transformed/sawing/file110.png  
  inflating: spectrograms/transformed/sawing/file125.png  
  inflating: spectrograms/transformed/sawing/file128.png  
  inflating: spectrograms/transformed/sawing/file151.png  
  inflating: spec

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
from PIL import Image

def crop_image(image_path, left, upper, right, lower, output_path):
    with Image.open(image_path) as img:
        cropped_img = img.crop((left, upper, right, lower))
        cropped_img.save(output_path)


In [ ]:
from PIL import Image

def threshold_image(image_path):
    img = Image.open(image_path)
    
    # img = img.convert('L')
    
    img = img.point(lambda p: 255 if p > 165 else p)
    img = img.point(lambda p: 110 if p < 130  else p)
    
    # Save the modified image
    img.save("thresholded_image.jpg")


In [ ]:
def pre_process_img(source_filepath, dest_filepath):

  #crop the image
  left, upper, right, lower = 80, 60, 475, 425
  with Image.open(source_filepath) as img:
      cropped_img = img.crop((left, upper, right, lower))
  
  #make it greyscale
  grey_img = cropped_img.convert('L')

  #pixel transform
  grey_img = grey_img.point(lambda p: 255 if p > 165 else p)
  grey_img = grey_img.point(lambda p: 110 if p < 130  else p)
  print(np.asarray(grey_img))
  #save image
  # grey_img.save(dest_filepath)
  # print(np.asarray(grey_img).shape)
  return np.asarray(grey_img)


In [ ]:

# pre_process_img('spectrograms/sawing/file1.png', 'file1.png')

[[[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [147 110 110 255]
  [147 110 110 255]
  [147 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [147 110 110 255]
  [147 110 110 255]
  [147 110 110 255]]

 ...

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [140 110 110 255]
  [140 110 110 255]
  [140 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [140 110 110 255]
  [140 110 110 255]
  [140 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 130 255]
  ...
  [255 110 110 255]
  [255 110 110 255]
  [255 110 110 255]]]
(365, 395, 4)


array([[[110, 110, 110, 255],
        [110, 110, 110, 255],
        [110, 110, 110, 255],
        ...,
        [110, 110, 110, 255],
        [110, 110, 110, 255],
        [110, 110, 110, 255]],

       [[110, 110, 110, 255],
        [110, 110, 110, 255],
        [110, 110, 110, 255],
        ...,
        [147, 110, 110, 255],
        [147, 110, 110, 255],
        [147, 110, 110, 255]],

       [[110, 110, 110, 255],
        [110, 110, 110, 255],
        [110, 110, 110, 255],
        ...,
        [147, 110, 110, 255],
        [147, 110, 110, 255],
        [147, 110, 110, 255]],

       ...,

       [[110, 110, 110, 255],
        [110, 110, 110, 255],
        [110, 110, 110, 255],
        ...,
        [140, 110, 110, 255],
        [140, 110, 110, 255],
        [140, 110, 110, 255]],

       [[110, 110, 110, 255],
        [110, 110, 110, 255],
        [110, 110, 110, 255],
        ...,
        [140, 110, 110, 255],
        [140, 110, 110, 255],
        [140, 110, 110, 255]],

       [[110

In [ ]:
# pre_process_img('spectrograms/ambient/file1.png', 'spectrograms/transformed/ambient/file1.png')

In [ ]:
# import multiprocessing as mp
# import os

# num_processes = mp.cpu_count()
# pool = mp.Pool(num_processes)

# out_dir = "spectrograms/transformed/ambient/"
# in_dir = "spectrograms/ambient/"

# if __name__ == '__main__':

#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir)

#     files = os.listdir(in_dir)

#     # Create a pool of processes
#     num_processes = mp.cpu_count()
#     pool = mp.Pool(num_processes)

#     # Use the pool to process the files
#     results = []
#     for f in files:
#         result = pool.apply_async(pre_process_img, args=(in_dir + f, out_dir + f))
#         results.append(result)

#     # Wait for all processes to complete
#     for result in results:
#         result.wait()

#     # Close the pool to free up resources
#     pool.close()
#     pool.join()

In [ ]:
# !zip -r "transformed.zip" "spectrograms/transformed/"

Streaming output truncated to the last 5000 lines.
  adding: spectrograms/transformed/sawing/file5452.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file2100.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file2287.png (deflated 10%)
  adding: spectrograms/transformed/sawing/file7303.png (deflated 10%)
  adding: spectrograms/transformed/sawing/file236.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file4116.png (deflated 10%)
  adding: spectrograms/transformed/sawing/file1952.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file3367.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file964.png (deflated 10%)
  adding: spectrograms/transformed/sawing/file885.png (deflated 10%)
  adding: spectrograms/transformed/sawing/file8266.png (deflated 10%)
  adding: spectrograms/transformed/sawing/file7719.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file5939.png (deflated 9%)
  adding: spectrograms/transformed/sawing/file849

In [ ]:
#cnn with 2 layers

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.callbacks import EarlyStopping

# Define the input shape
# Define the input shape
input_shape = (365, 395, 1)

# Define the model
model = Sequential()

# Add a convolutional layer with 16 filters and a 3x3 kernel size
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Flatten the output of the convolutional layer
model.add(Flatten())

# Add the output layer with 2 neurons for the 2 classes (sawing and not sawing)
model.add(Dense(2, activation='softmax'))

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()


# Define the data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the training data from the directory
train_data = data_generator.flow_from_directory(
    'spectrograms/transformed/',
    target_size=(365, 395),
    batch_size=256,
    color_mode='grayscale',
    class_mode='categorical'
)

early_stop = EarlyStopping(monitor='loss', patience=3)

# Train the model for 10 epochs
model.fit(train_data, epochs=3, callbacks=[early_stop])



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 363, 393, 16)      160       
                                                                 
 flatten_2 (Flatten)         (None, 2282544)           0         
                                                                 
 dense_2 (Dense)             (None, 2)                 4565090   
                                                                 
Total params: 4,565,250
Trainable params: 4,565,250
Non-trainable params: 0
_________________________________________________________________
Found 20000 images belonging to 2 classes.
Epoch 1/3
79/79 [==============================] - 1084s 14s/step - loss: 3.6520 - accuracy: 0.9536
Epoch 2/3
79/79 [==============================] - 1059s 13s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
79/79 [==============================] - 1065s 13s/

In [ ]:
model.save('cnn_model_v1.h5')

In [ ]:
from keras.models import load_model
from PIL import Image
import numpy as np
from tensorflow.keras.utils import load_img
import tensorflow as tf

# Load the saved model from a file
loaded_model = load_model('cnn_model_v1.h5')

pre_process_img("spectrograms/sawing/file112.png", "test7.png")


# Load the image
img = tf.keras.utils.load_img('test7.png', target_size=(365, 395))


# Convert the image to a numpy array
# x = image.img_to_array(img)
x = np.asarray(img)
print(x)
print('*'*10)

# Preprocess the image using the same method as the training data
x = x / 255.

# Reshape the input data to match the expected shape of the model
x = np.expand_dims(x, axis=0)
# print("x", x.shape)

# Make the prediction using the model
preds = model.predict(x)

# Print the predicted class
print(preds.argmax())

if preds.argmax() == 1:
  print('sawing!!')
else:
  print('not sawing!')


[[[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [147 110 110 255]
  [147 110 110 255]
  [147 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [147 110 110 255]
  [147 110 110 255]
  [147 110 110 255]]

 ...

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [140 110 110 255]
  [140 110 110 255]
  [140 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 110 255]
  ...
  [140 110 110 255]
  [140 110 110 255]
  [140 110 110 255]]

 [[110 110 110 255]
  [110 110 110 255]
  [110 110 130 255]
  ...
  [255 110 110 255]
  [255 110 110 255]
  [255 110 110 255]]]
[[[110 110 110]
  [110 110 110]
  [110 110 110]
  ...
  [110 110 110]
  [110 110 110]
  [110 110 110]]

 [[110 110 110]
  [110 110 110]
  [110 110 110]
  ...
  [147 110 110]
  [147 110 110]
  [147 110 110]]

 [[110 110 110]
  

In [ ]:
def pre_process_img2(source_filepath):

  #crop the image
  left, upper, right, lower = 80, 60, 475, 425
  with Image.open(source_filepath) as img:
      cropped_img = img.crop((left, upper, right, lower))
  
  #make it greyscale
  grey_img = cropped_img.convert('L')

  #pixel transform
  grey_img = grey_img.point(lambda p: 255 if p > 165 else p)
  grey_img = grey_img.point(lambda p: 110 if p < 130  else p)

  return np.asarray(grey_img)


In [ ]:
#libraries
from keras.models import load_model
from PIL import Image
import numpy as np
# from tensorflow.keras.utils import load_img
import tensorflow as tf


def predict_spectrogram(filepath):
  model = load_model('/content/drive/MyDrive/Datasets/mms/cnn_model_v1.hdf5')
  img_data = pre_process_img2(filepath)
  x = img_data / 255.

  x = np.expand_dims(x, axis=0)

  preds = model.predict(x)
  return True if preds.argmax()==1 else False
predict_spectrogram('/content/drive/MyDrive/Datasets/spectogramImgs/file1.png')

1/1 [==============================] - 0s 419ms/step


True

In [ ]:
!pip show tensorflow
!pip show keras

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl
Name: keras
Version: 2.12.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: tensorflow


In [ ]:
!pip show flatbuffers tensorboard 

Name: flatbuffers
Version: 23.3.3
Summary: The FlatBuffers serialization format for Python
Home-page: https://google.github.io/flatbuffers/
Author: Derek Bailey
Author-email: derekbailey@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: tensorflow
---
Name: tensorboard
Version: 2.12.2
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, google-auth, google-auth-oauthlib, grpcio, markdown, numpy, protobuf, requests, setuptools, tensorboard-data-server, tensorboard-plugin-wit, werkzeug, wheel
Required-by: tensorflow


In [ ]:
!pip show protobuf

Name: protobuf
Version: 3.20.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: google-api-core, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-datastore, google-cloud-firestore, google-cloud-language, google-cloud-translate, googleapis-common-protos, grpcio-status, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata


In [ ]:
!pip show Pillow

Name: Pillow
Version: 8.4.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Alex Clark (PIL Fork Author)
Author-email: aclark@python-pillow.org
License: HPND
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: bokeh, dopamine-rl, fastai, imageio, imgaug, matplotlib, scikit-image, torchvision, wordcloud


In [ ]:
!pip show numpy

Name: numpy
Version: 1.22.4
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: albumentations, altair, arviz, astropy, autograd, blis, bokeh, chex, cmdstanpy, contourpy, cufflinks, cvxpy, datascience, db-dtypes, dopamine-rl, ecos, flax, folium, gensim, gym, h5py, holoviews, hyperopt, imageio, imbalanced-learn, imgaug, jax, jaxlib, librosa, lightgbm, matplotlib, matplotlib-venn, missingno, mizani, ml-dtypes, mlxtend, moviepy, music21, nibabel, numba, numexpr, opencv-contrib-python, opencv-python, opencv-python-headless, opt-einsum, optax, orbax-checkpoint, osqp, pandas, pandas-gbq, patsy, plotnine, prophet, pyarrow, pycocotools, pyerfa, pymc, pytensor, python-louvain, PyWavelets, qdldl, qudida, scikit-image, scikit-learn, scipy, scs, seaborn, shapely, sklearn-pandas, soxr, spacy, statsmodels, tabl

In [ ]:
!unzip "/content/drive/MyDrive/Datasets/mms/spectrograms.zip"

Streaming output truncated to the last 5000 lines.
  inflating: spectrograms/transformed/sawing/file30.png  
  inflating: spectrograms/transformed/sawing/file35.png  
  inflating: spectrograms/transformed/sawing/file38.png  
  inflating: spectrograms/transformed/sawing/file54.png  
  inflating: spectrograms/transformed/sawing/file59.png  
  inflating: spectrograms/transformed/sawing/file62.png  
  inflating: spectrograms/transformed/sawing/file66.png  
  inflating: spectrograms/transformed/sawing/file75.png  
  inflating: spectrograms/transformed/sawing/file82.png  
  inflating: spectrograms/transformed/sawing/file85.png  
  inflating: spectrograms/transformed/sawing/file93.png  
  inflating: spectrograms/transformed/sawing/file96.png  
  inflating: spectrograms/transformed/sawing/file110.png  
  inflating: spectrograms/transformed/sawing/file125.png  
  inflating: spectrograms/transformed/sawing/file128.png  
  inflating: spectrograms/transformed/sawing/file151.png  
  inflating: spec

In [ ]:
import os
os.makedirs('spectrograms/transformed-v3/sawing/')

In [ ]:
import os
os.makedirs('spectrograms/transformed-v3/ambient/')

*************************************


In [ ]:
from PIL import Image
import numpy as np

def pre_process_img_v3(source_filepath,f):

    # crop the image
    left, upper, right, lower = 170, 200, 350, 380
    with Image.open(source_filepath) as img:
        cropped_img = img.crop((left, upper, right, lower))

    # make it greyscale
    grey_img = cropped_img.convert('L')

    # pixel transform
    grey_img = grey_img.point(lambda p: 255 if p > 165 else p)
    grey_img = grey_img.point(lambda p: 110 if p < 130 else p)

    # convert to numpy array 
    np_img = np.asarray(grey_img)

    # Image.fromarray(np_img.save('random-test.png'))
    # img = Image.fromarray(np_img)
    # img.save(f)

    return np_img
# pre_process_img_v3('/content/spectrograms/sawing/file1090.png', 'random112.png')

In [ ]:
print(len(os.listdir('spectrograms/transformed-v3/ambient/')))

0


In [ ]:

import multiprocessing as mp
import os

num_processes = mp.cpu_count()
pool = mp.Pool(num_processes)

out_dir = "spectrograms/transformed-v3/sawing/"
in_dir = "spectrograms/sawing/"

if __name__ == '__main__':

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    files = os.listdir(in_dir)

    # Create a pool of processes
    num_processes = mp.cpu_count()
    pool = mp.Pool(num_processes)

    # Use the pool to process the files
    results = []
    for f in files:
        result = pool.apply_async(pre_process_img_v3, args=(in_dir + f, out_dir + f))
        results.append(result)

    # Wait for all processes to complete
    for result in results:
        result.wait()

    # Close the pool to free up resources
    pool.close()
    pool.join()

In [ ]:
print(len(os.listdir('spectrograms/transformed-v3/sawing/')))

10000


In [ ]:
import multiprocessing as mp
import os

num_processes = mp.cpu_count()
pool = mp.Pool(num_processes)

out_dir = "spectrograms/transformed-v3/ambient/"
in_dir = "spectrograms/ambient/"

if __name__ == '__main__':

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    files = os.listdir(in_dir)

    # Create a pool of processes
    num_processes = mp.cpu_count()
    pool = mp.Pool(num_processes)

    # Use the pool to process the files
    results = []
    for f in files:
        result = pool.apply_async(pre_process_img_v3, args=(in_dir + f, out_dir + f))
        results.append(result)

    # Wait for all processes to complete
    for result in results:
        result.wait()

    # Close the pool to free up resources
    pool.close()
    pool.join()

In [ ]:
print(len(os.listdir('/content/spectrograms/transformed-v3/ambient')))

10000


In [ ]:
# cnn model v3 even lightweight

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adagrad

# Define the input shape
input_shape = (180, 180, 1)

# Define the model
model = tf.keras.Sequential([
    Conv2D(4, kernel_size=(2, 2), activation=Activation('swish'), input_shape=input_shape),
    Flatten(),
    Dense(2, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and Adagrad optimizer
model.compile(loss='binary_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])

# Define the data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the training data from the directory
train_data = data_generator.flow_from_directory(
    'spectrograms/transformed-v3/',
    target_size=(180,180),
    batch_size=256,
    color_mode='grayscale',
    class_mode='categorical'
)

# print(train_data)

# Train the model for 3 epochs
model.fit(train_data, epochs=3)

# Convert the model to TensorFlow Lite format with float16 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
quantized_model = converter.convert()

# Save the quantized model
with open('cnn_model_v2.tflite', 'wb') as f:
    f.write(quantized_model)


Found 20000 images belonging to 2 classes.
Epoch 1/3
79/79 [==============================] - 24s 161ms/step - loss: 0.1407 - accuracy: 0.9607
Epoch 2/3
79/79 [==============================] - 12s 156ms/step - loss: 0.0227 - accuracy: 1.0000
Epoch 3/3
79/79 [==============================] - 13s 159ms/step - loss: 0.0127 - accuracy: 1.0000


In [ ]:
import tensorflow as tf
import numpy as np
# from PIL import Image

def predict_with_tflite(image_path, model_path):

    image = pre_process_img_v3(image_path,'')

    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=-1)
    image = image.astype(np.float32)

    # print(image.shape)

    # Load the model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get the input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make the prediction
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Return the prediction
    return prediction[0]


prediction = predict_with_tflite('spectrograms/sawing/file7810.png', 'cnn_model_v2.tflite')
prediction_dic = {0:'ambient',1:'sawing'}
print(prediction_dic[np.argmax(prediction)])

[0.00964801 0.99143595]
sawing
